https://stepik.org/lesson/1576188/step/7

In [ ]:
import torch
import torch.nn as nn

class NNModel(nn.Module):
    def __init__(self, output_size):
        super(NNModel, self).__init__()
        self.net1 = nn.Sequential(
            # Вход: (batch_size, 3, 16, 16)
            nn.Conv2d(3, 16, (3, 3), stride=1, padding=1, bias=True),  # (batch_size, 16, 16, 16)
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 2), stride=2),                            # (batch_size, 16, 8, 8)
            nn.Conv2d(16, 32, (3, 3), stride=1, padding=1, bias=True), # (batch_size, 32, 8, 8)
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 2), stride=2),                            # (batch_size, 32, 4, 4)
            nn.Flatten()                    # (batch_size, 32 * 4 * 4) = (batch_size, 512)
        )

        self.net2 = nn.Sequential(
            nn.Linear(12, 64, bias=False),  # (batch_size, 64)
            nn.Sigmoid(),
            nn.BatchNorm1d(64)
        )

        self.output = nn.Linear(512 + 64, output_size, bias=True)

    def forward(self, x_img, x):
        x_img = self.net1(x_img)          # (batch_size, 512)
        x = self.net2(x)                  # (batch_size, 64)
        x = torch.cat([x_img, x], dim=1)  # (batch_size, 512 + 64 = 576)

        return self.output(x)             # (batch_size, output_size)


# тензоры data_img, data_x в программе не менять
batch_size = 32
data_img = torch.rand(batch_size, 3, 16, 16)
data_x = torch.rand(batch_size, 12)

model = NNModel(output_size=10)
model.eval()
predict = model(data_img, data_x)
# print(predict.shape) # (32, 10)

torch.Size([32, 10])


# Размерность после сверточных слоев (net1)
Сеть net1 принимает на вход изображение размером `(batch_size, 3, 16, 16)`. Посмотрим, как изменяется размерность на каждом слое:

1. Conv2d`(3, 16, 3, stride=1, padding=1)`

Вход: `(batch_size, 3, 16, 16)`

Выход: `(batch_size, 16, 16, 16)` (размер не изменился, так как stride=1 и padding=1)

2. MaxPool2d(2, stride=2)

Выход: `(batch_size, 16, 8, 8)`

3. Conv2d(16, 32, 3, stride=1, padding=1)

Выход: `(batch_size, 32, 8, 8)`

4. MaxPool2d(2, stride=2)

Выход: `(batch_size, 32, 4, 4)`

5. Flatten()

Выход: `(batch_size, 32 * 4 * 4) = (batch_size, 512)`

Таким образом, net1 выдаёт x_img размерности `(batch_size, 512)`.